# Loading WHO data with dlt

In [ ]:
import dlt

In [ ]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

maximum_page = 3000

BASE_WHO_URL = "https://ghoapi.azureedge.net/api/"
ENDPOINTS= {"dimensions": "Dimension", "indicators": "Indicator"}
            # "hospital_beds": "WHS6_102", "hospitals_per_100k": "DEVICES00"}

# Define the API resource for WHO data
def fetch_who_data(endpoint, params={}):
    client = RESTClient(
        base_url=BASE_WHO_URL,
        paginator=PageNumberPaginator(
            base_page=0,
            total_path=None,
            maximum_page=1
        )
    )

    for page in client.paginate(endpoint, params=params):    # <--- API endpoint for retrieving taxi ride data
        yield page   # <--- yield data to manage memory

@dlt.source
def who_source():
    for endpoint_name, endpoint in ENDPOINTS.items():
        params={"size":100}
        yield dlt.resource(
            fetch_who_data(endpoint, params),
            name=endpoint_name,
            write_disposition="replace",
        )
@dlt.source()
def all_indicators():    
    for endpoint, endpoint_name in pipeline.dataset().indicators.select("indicator_code", "indicator_name").df().values[0:100]:
        print(f"ingesting source {endpoint_name}")
        yield dlt.resource(
            fetch_who_data(endpoint),
            name=endpoint,
            write_disposition="replace",
        )

# define new dlt pipeline
pipeline = dlt.pipeline(pipeline_name="who_source", destination="duckdb", dataset_name="who_data")

# run the pipeline with the new resource
load_info = pipeline.run(who_source())
row_counts = pipeline.last_trace.last_normalize_info

In [ ]:
pipeline.last_trace.last_normalize_info

In [ ]:
pipeline.run(all_indicators())

In [ ]:
for x,y in  pipeline.dataset().indicators.select("indicator_code", "indicator_name").df().values[0:10]:
    print(f"name {x}, table {y}")

In [ ]:
pipeline.dataset_name

In [ ]:
# Start a connection to your database using native `duckdb` connection and look what tables were generated:

import duckdb

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").select("name").df().values

In [ ]:
conn.sql("select * from air_42")

In [ ]:
conn.sql()